In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random

In [4]:
FIGURES_PATH = 'out/figures'
DATASETS_PATH = 'out/datasets'

In [5]:
data = pd.read_csv('datasets/receipts_new.csv')
data = data.rename(columns = {'line_item_id': 'product_id'})
data

,gid,transaction_key,store_id,opened_date,product_id,line_quantity,line_item_price,line_item_cost,line_margin,line_type
0,BipqVCA9TxmTukbRjRKJXg,009_002_74_2022-04-18 12:22:18_9058,2,2022-04-18,83591944,1.0,2990.0,2516.10,-24.43,Sales
1,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,15058563,2.0,1413.0,1206.73,412.54,Undefined
2,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,1.0,338.0,179.51,158.49,Undefined
3,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,3.0,338.0,179.51,475.47,Undefined
4,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,16042299,1.0,1634.0,1402.83,231.17,Undefined
...,...,...,...,...,...,...,...,...,...,...
27601037,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,21.0,43.0,44.66,-185.36,Sales
27601038,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,1.0,43.0,44.66,-8.83,Sales
27601039,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,82258971,1.0,950.0,969.86,-178.19,Sales
27601040,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,49000003,1.0,4.0,NaN,NaN,Sales


Можно заметить, что совпадают некоторые строки по ключевому признаку - transaction_key, необходимо произвести схлопывание данных по одинаковым ключам с суммированием признаков line_quantity.

### Подключение категорий товаров

In [6]:
cats = pd.read_csv('datasets/ProductsModels.csv')
cats = cats.rename(columns = {'model_id': 'category_id', 'name': 'category_name'})
cats

,product_id,category_id,category_name
0,18843508,201658,Ручной инструмент для обработки почвы: культив...
1,82406527,201271,Комнатные растения
2,82597131,201274,Саженцы плодовых деревьев и кустарников
3,84738086,200990,Шторка для ванной комнаты
4,18816091,200340,Кабели для ТВ
...,...,...,...
540648,82192555,202275,Наконечник для карниза
540649,82192571,202275,Наконечник для карниза
540650,82088853,202276,Кронштейн для штанги со сверлением
540651,81966519,202346,Ткани и тюли на отрез


In [7]:
cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540653 entries, 0 to 540652
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   product_id     540653 non-null  int64 
 1   category_id    540653 non-null  int64 
 2   category_name  540627 non-null  object
dtypes: int64(2), object(1)
memory usage: 12.4+ MB


In [8]:
data = data.join(cats.set_index('product_id'), on='product_id', how='left')
data

,gid,transaction_key,store_id,opened_date,product_id,line_quantity,line_item_price,line_item_cost,line_margin,line_type,category_id,category_name
0,BipqVCA9TxmTukbRjRKJXg,009_002_74_2022-04-18 12:22:18_9058,2,2022-04-18,83591944,1.0,2990.0,2516.10,-24.43,Sales,201981.0,Стремянки
1,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,15058563,2.0,1413.0,1206.73,412.54,Undefined,202253.0,Шпаклевка
2,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,1.0,338.0,179.51,158.49,Undefined,201089.0,Фитинги латунные для полиэтиленовых труб
3,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,3.0,338.0,179.51,475.47,Undefined,201089.0,Фитинги латунные для полиэтиленовых труб
4,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,16042299,1.0,1634.0,1402.83,231.17,Undefined,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27601037,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,21.0,43.0,44.66,-185.36,Sales,200308.0,Силовые розетки и вилки
27601038,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,1.0,43.0,44.66,-8.83,Sales,200308.0,Силовые розетки и вилки
27601039,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,82258971,1.0,950.0,969.86,-178.19,Sales,200337.0,Силовые кабели
27601040,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,49000003,1.0,4.0,NaN,NaN,Sales,NaN,NaN


### Добавление дней недели

In [9]:
def get_data(str):
    return str.split('_')[3:4][0]

dates = data['transaction_key'].apply(get_data)
dates = pd.to_datetime(dates)
data['datetime'] = dates
data

,gid,transaction_key,store_id,opened_date,product_id,line_quantity,line_item_price,line_item_cost,line_margin,line_type,category_id,category_name,datetime
0,BipqVCA9TxmTukbRjRKJXg,009_002_74_2022-04-18 12:22:18_9058,2,2022-04-18,83591944,1.0,2990.0,2516.10,-24.43,Sales,201981.0,Стремянки,2022-04-18 12:22:18
1,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,15058563,2.0,1413.0,1206.73,412.54,Undefined,202253.0,Шпаклевка,2022-08-26 18:00:29
2,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,1.0,338.0,179.51,158.49,Undefined,201089.0,Фитинги латунные для полиэтиленовых труб,2022-08-26 18:00:29
3,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,3.0,338.0,179.51,475.47,Undefined,201089.0,Фитинги латунные для полиэтиленовых труб,2022-08-26 18:00:29
4,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,16042299,1.0,1634.0,1402.83,231.17,Undefined,NaN,NaN,2022-08-26 18:00:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27601037,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,21.0,43.0,44.66,-185.36,Sales,200308.0,Силовые розетки и вилки,2022-09-08 11:14:12
27601038,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,1.0,43.0,44.66,-8.83,Sales,200308.0,Силовые розетки и вилки,2022-09-08 11:14:12
27601039,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,82258971,1.0,950.0,969.86,-178.19,Sales,200337.0,Силовые кабели,2022-09-08 11:14:12
27601040,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,49000003,1.0,4.0,NaN,NaN,Sales,NaN,NaN,2022-09-08 11:14:12


In [ ]:
# from datetime import datetime

def get_weekday(datetime):
    return datetime.weekday()

weekdays = data['datetime'].apply(get_weekday)
data['weekday'] = weekdays
data

### Схлопывание одинаковых позиций в чеках

In [ ]:
# data_grouped = data.groupby(by = ['transaction_key'])
# data_grouped.head(5)

### Сохранение обработанных данных

In [ ]:
data.to_csv(DATASETS_PATH + 'data_processed.csv')